In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# initializes the OpenAI API key and defines the model to be used.

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# defines the Website class for fetching and parsing webpage content

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
   
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
# This cell tests the Website class with a sample URL and prints its contents and links

test_david = Website("https://satoshilabs.com/")

print(test_david.get_contents())

test_david.links

Webpage Title:
Empowering Independence | SatoshiLabs
Webpage Contents:
Scroll down
Manifesto
Careers
Our story
Newsroom
Reach out
Careers
Our story
Our companies
Manifesto
Newsroom
Reach out
Proud to be a bitcoin-first tech holding, fighting the good fight for self-sovereignty, top-notch digital security, and decentralized finance.
Empowering Independence
Self-owned & independent
10+ years of innovation
250+ rebels & creators
Join us
Without the freedom to transact, we have no power to secure our rights. That's why we're here - to help you grab hold of your financial independence and digital security.
We’re a Czech technology holding creating HW and SW innovations in crypto and open source.
Read the Manifesto
Meet the SatoshiLabs Group
The inventor of the world's first hardware wallet for secure asset storage and protection.
Your friend in crypto, helping newcomers buy, sell, and save bitcoin safely.
The creator of the first secure auditable chip design redefining hardware security.
A 

['/manifesto',
 '/',
 '/careers',
 '/our-story',
 '/newsroom',
 '/reach-out',
 '/careers',
 '/our-story',
 'https://satoshilabs.com/#our-companies',
 '/manifesto',
 '/newsroom',
 '/reach-out',
 '/careers',
 '/manifesto',
 'https://trezor.io/',
 'https://invity.io/',
 'https://tropicsquare.com/',
 'https://vexl.it/',
 'https://github.com/trezor',
 '/our-story',
 'https://www.satoshilabs.com/presskit',
 '/news/vexl-foundation-launches-freedom-store---its-own-mobile-app-marketplace',
 'https://freedomstore.io/',
 'https://freedomstore.io/',
 'https://www.vexl.foundation/',
 '/news/jan-pleskac-steps-back-into-leadership-at-tropic-square',
 'http://www.tropicsquare.com/',
 'mailto:pr@tropicsquare.com',
 '/news/tropic-square-secures-double-win-at-embedded-world-2025',
 'https://www.tropicsquare.com/tropic01',
 'http://www.tropicsquare.com/',
 'mailto:pr@tropicsquare.com',
 '/news/tropic-square-announces-official-launch-general-availability-of-tropic01',
 'https://www.tropicsquare.com/tropic0

In [5]:
# the system prompt for the LLM to extract relevant links from a webpage
 
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [7]:
# function to generate a user prompt for the LLM to get relevant links

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(test_david))

Here is the list of links on the website of https://satoshilabs.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/manifesto
/
/careers
/our-story
/newsroom
/reach-out
/careers
/our-story
https://satoshilabs.com/#our-companies
/manifesto
/newsroom
/reach-out
/careers
/manifesto
https://trezor.io/
https://invity.io/
https://tropicsquare.com/
https://vexl.it/
https://github.com/trezor
/our-story
https://www.satoshilabs.com/presskit
/news/vexl-foundation-launches-freedom-store---its-own-mobile-app-marketplace
https://freedomstore.io/
https://freedomstore.io/
https://www.vexl.foundation/
/news/jan-pleskac-steps-back-into-leadership-at-tropic-square
http://www.tropicsquare.com/
mailto:pr@tropicsquare.com
/news/tropic-square-secures-double-win-at-embedded-world-2025
https://www.tropicsquare.com/tropic01
http://

In [9]:
# call the LLM to extract relevant links from a given URL

def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# tests the Website class with satoshilabs and displays its links

satoshi_labs_website_test = Website("https://satoshilabs.com/")
satoshi_labs_website_test.links

['/manifesto',
 '/',
 '/careers',
 '/our-story',
 '/newsroom',
 '/reach-out',
 '/careers',
 '/our-story',
 'https://satoshilabs.com/#our-companies',
 '/manifesto',
 '/newsroom',
 '/reach-out',
 '/careers',
 '/manifesto',
 'https://trezor.io/',
 'https://invity.io/',
 'https://tropicsquare.com/',
 'https://vexl.it/',
 'https://github.com/trezor',
 '/our-story',
 'https://www.satoshilabs.com/presskit',
 '/news/vexl-foundation-launches-freedom-store---its-own-mobile-app-marketplace',
 'https://freedomstore.io/',
 'https://freedomstore.io/',
 'https://www.vexl.foundation/',
 '/news/jan-pleskac-steps-back-into-leadership-at-tropic-square',
 'http://www.tropicsquare.com/',
 'mailto:pr@tropicsquare.com',
 '/news/tropic-square-secures-double-win-at-embedded-world-2025',
 'https://www.tropicsquare.com/tropic01',
 'http://www.tropicsquare.com/',
 'mailto:pr@tropicsquare.com',
 '/news/tropic-square-announces-official-launch-general-availability-of-tropic01',
 'https://www.tropicsquare.com/tropic0

In [11]:
get_links("https://satoshilabs.com/")

{'links': [{'type': 'about page', 'url': 'https://satoshilabs.com/our-story'},
  {'type': 'careers page', 'url': 'https://satoshilabs.com/careers'},
  {'type': 'press kit', 'url': 'https://www.satoshilabs.com/presskit'},
  {'type': 'newsroom', 'url': 'https://satoshilabs.com/newsroom'},
  {'type': 'manifesto', 'url': 'https://satoshilabs.com/manifesto'}]}

In [12]:
# function to gather all relevant details from a website and its sub-pages
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
# get_all_details function with satoshilabs

print(get_all_details("https://satoshilabs.com/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://satoshilabs.com/our-story'}, {'type': 'careers page', 'url': 'https://satoshilabs.com/careers'}, {'type': 'newsroom', 'url': 'https://satoshilabs.com/newsroom'}, {'type': 'press kit', 'url': 'https://www.satoshilabs.com/presskit'}, {'type': 'manifesto', 'url': 'https://satoshilabs.com/manifesto'}]}
Landing page:
Webpage Title:
Empowering Independence | SatoshiLabs
Webpage Contents:
Scroll down
Manifesto
Careers
Our story
Newsroom
Reach out
Careers
Our story
Our companies
Manifesto
Newsroom
Reach out
Proud to be a bitcoin-first tech holding, fighting the good fight for self-sovereignty, top-notch digital security, and decentralized finance.
Empowering Independence
Self-owned & independent
10+ years of innovation
250+ rebels & creators
Join us
Without the freedom to transact, we have no power to secure our rights. That's why we're here - to help you grab hold of your financial independence and digital security.
We’re a Czech 

In [14]:
# defines the system prompt for the LLM to create a company brochure / i want respond in CZECH

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information.Please respond in Czech language."


In [15]:
# function to generate a user prompt for the LLM to create a brochure

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("SatoshiLabs", "https://satoshilabs.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://satoshilabs.com/our-story'}, {'type': 'careers page', 'url': 'https://satoshilabs.com/careers'}, {'type': 'press kit', 'url': 'https://www.satoshilabs.com/presskit'}, {'type': 'manifesto page', 'url': 'https://satoshilabs.com/manifesto'}, {'type': 'newsroom', 'url': 'https://satoshilabs.com/newsroom'}]}


'You are looking at a company called: SatoshiLabs\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nEmpowering Independence | SatoshiLabs\nWebpage Contents:\nScroll down\nManifesto\nCareers\nOur story\nNewsroom\nReach out\nCareers\nOur story\nOur companies\nManifesto\nNewsroom\nReach out\nProud to be a bitcoin-first tech holding, fighting the good fight for self-sovereignty, top-notch digital security, and decentralized finance.\nEmpowering Independence\nSelf-owned & independent\n10+ years of innovation\n250+ rebels & creators\nJoin us\nWithout the freedom to transact, we have no power to secure our rights. That\'s why we\'re here - to help you grab hold of your financial independence and digital security.\nWe’re a Czech technology holding creating HW and SW innovations in crypto and open source.\nRead the Manifesto\nMeet the SatoshiLabs Group\nThe inventor of t

In [17]:
# function to create and display a company brochure using the LLM

def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("SatoshiLabs", "https://satoshilabs.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://satoshilabs.com/our-story'}, {'type': 'careers page', 'url': 'https://satoshilabs.com/careers'}, {'type': 'newsroom page', 'url': 'https://satoshilabs.com/newsroom'}, {'type': 'manifesto page', 'url': 'https://satoshilabs.com/manifesto'}, {'type': 'press kit', 'url': 'https://www.satoshilabs.com/presskit'}]}


# SatoshiLabs Brožura

## O nás
SatoshiLabs je česká technologická holdingová společnost, která se specializuje na inovace v oblasti kryptoměn a otevřeného zdroje. Jsme pyšní na to, že jsme prvním technologickým holdingem zaměřeným na Bitcoin, který se zasazuje o sebedůvěru, špičkovou digitální bezpečnost a decentralizované finance.

## Naše hodnoty
- **Silná nezávislost**: Věříme v právo jednotlivce na finanční nezávislost a digitální bezpečnost.
- **Inovace**: Více než 10 let průkopnických řešení a inovací.
- **Komunita**: Naše rozsáhlá síť zahrnuje přes 250 rebelů a tvůrců, kteří spolupracují na vytváření revolučních řešení.

## Naše služby
- **Trezor**: Vynález prvního hardwarového peněženka na světě pro bezpečné ukládání a ochranu aktiv.
- **Slush Pool**: První bitcoinový těžební pool, který zlepšuje efektivitu a bezpečnost sítě.
- **Vexl**: Mobilní aplikace pro bezpečný P2P obchod s Bitcoinem bez nutnosti KYC.
- **Freedom Store**: Mobilní trh pro uživatele iOS, zaměřený na otevřené aplikace a řešení e-cash.

## Naše zákazníci
S více než 1,5 milionu zákazníků a uživatelů v 200+ zemích pokrýváme značný globální trh. Naši uživatelé hledají bezpečné a efektivní způsoby správy svých kryptoměnových aktiv.

## Kariéra u SatoshiLabs
Hledáme vášnivé a talentované jednotlivce, kteří se chtějí podílet na transformaci kryptoměnového průmyslu. U nás najdete:
- **Podporující kulturu**: Skvělý tým plný ztělesnění inovace a spolupráce.
- **Možnost růstu**: V SatoshiLabs máte příležitost se neustále učit a rozvíjet své dovednosti.
- **Otevřenost k novým myšlenkám**: Vítáme nové nápady a iniciativy.

## Kontaktujte nás
Zajímáte se o spolupráci, investice nebo práci v SatoshiLabs? Neváhejte nás kontaktovat prostřednictvím našich webových stránek a připojte se k nám na cestě k digitální svobodě a nezávislosti.

---

SatoshiLabs – Posilujeme nezávislost a měníme budoucnost digitálních financí!

In [19]:
# function to stream the creation and display of a company brochure

def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
# streams the creation and display of a brochure for SatoshiLabs

stream_brochure("SatoshiLabs", "https://satoshilabs.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://satoshilabs.com/our-story'}, {'type': 'careers page', 'url': 'https://satoshilabs.com/careers'}, {'type': 'press kit', 'url': 'https://www.satoshilabs.com/presskit'}, {'type': 'manifesto', 'url': 'https://satoshilabs.com/manifesto'}, {'type': 'newsroom', 'url': 'https://satoshilabs.com/newsroom'}]}


# Brožura společnosti SatoshiLabs

## O nás
SatoshiLabs je český technologický holding specializovaný na inovace v oblasti hardware a software, které se zaměřují na kryptoměny a open source. Jsme hrdí na to, že jsme bitcoin-first firma bojující za individuální svobodu, digitální bezpečnost a decentralizované finance.

### Naše mise
Bez svobody transakce nemáme moc zajistit si své práva. Jsme tu, abychom vám pomohli získat finanční nezávislost a digitální bezpečnost. Od roku 2013 jsme přispěli více než 10 lety inovací, s 250+ revolučními tvůrci a více než 1,5 miliony spokojených zákazníků po celém světě.

## Naše produkty
- **Trezor**: První hardware peněženka na světě pro zabezpečené ukládání a ochranu digitálních aktiv.
- **Technologie Shamir Backup**: Revoluční způsob vytváření, používání a ochrany záloh vašich klíčů.
- **Trezor Blockbook**: Backend služba poskytující úplný index blockchainu pro nalezení informací o transakcích.
- **Slush Pool**: První bitcoinová těžební pool, která zlepšuje efektivitu sítě a zajišťuje bezpečnost.

## Naše kultury
Naše kultura je postavena na hodnotách svobody, inovací a spolupráce. Věříme v sílu decentralizace a snažíme se vytvořit pracovní prostředí, kde každý má možnost přispět k našim inovacím. Naši zaměstnanci, označováni jako "rebels" a "creators", hrají klíčovou roli v našem úsilí o změnu světa. 

### Hlas z vedení
"I don't need to change the world, but I like the fact our team is doing exactly that," říká Stepan Uherik, CFO Satoshi

KeyboardInterrupt: 